In [441]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def is_close(self, other, tolerance):
        return abs(self.x - other.x) < tolerance and abs(self.y - other.y) < tolerance

In [442]:
class Edge:
    def __init__(self, p1, p2, x_speed=0, y_speed=0):
        self.p1 = p1
        self.p2 = p2
        self.x_speed = x_speed
        self.y_speed = y_speed

    def is_point_on_edge(self, point):
        return point.x >= min(self.p1.x, self.p2.x) and point.x <= max(self.p1.x, self.p2.x) and point.y >= min(self.p1.y, self.p2.y) and point.y <= max(self.p1.y, self.p2.y)

In [443]:
class Car:
    def __init__(self, name, position, source, destination, spawn_time, speed = 1):
        self.name = name
        self.position = position
        self.visited_intersection = False
        self.source = source
        self.destination = destination
        self.speed = speed
        self.spawn_time = spawn_time

    def move(self, x, y):
        self.position.x += x
        self.position.y += y

    def is_at_intersection(self):
        if (self.position.x == 0 and self.position.y > -1 and self.position.y < 1) \
            or (self.position.y == 0 and self.position.x > -1 and self.position.x < 1):
            if not self.visited_intersection:
                self.visited_intersection = True
                if self.destination == 'E':
                    self.position.x = 1
                    self.position.y = 0
                elif self.destination == 'S':
                    self.position.x = 1
                    self.position.y = -1
            return True
        return False

    def distance_to_intersection(self):
        return abs(self.position.x) + abs(self.position.y)

In [444]:
import pandas as pd

class Controller:
    def __init__(self, cars, edges, speed_reduction_factor=0.5):
        self.cars = cars
        self.edges = edges
        self.time = 0
        self.intersection_block_times = {}
        self.speed_reduction_factor = speed_reduction_factor
        self.edges_with_reduced_speed = []
        self.log_content = []
        self.active_cars = []
        self.car_positions = {}
        self.no_of_cars = len(cars)
        self.cars_passed_intersection = []
        # print("initialized with srf", speed_reduction_factor)

    def check_for_collision(self):
        for car1 in self.active_cars:
            for car2 in self.active_cars:
                if car1.name == car2.name:
                    continue
                if car1.position.is_close(car2.position, 1):
                    # print(f"Collision detected between {car1.name} and {car2.name}")
                    self.log_content.append(f"{self.time} Collision detected between {car1.name} and {car2.name}")
                    return True

    def predict_collision_at_intersection(self, car):
        car_edge = self.get_edge(car)
        # print(car_edge.x_speed, car_edge.y_speed)
        time_to_intersection = self.time + (abs(car.position.x) + abs(car.position.y) ) / max(abs(car_edge.x_speed), abs(car_edge.y_speed))
        self.log_content.append(f"{self.time} car {car.name} time to intersection {time_to_intersection}")
        # print(car.name, "tti", time_to_intersection, self.speed_reduction_factor)
        for car_name in self.intersection_block_times:
            if car_name == car.name:
                continue
            if car_name in self.cars_passed_intersection or car.name in self.cars_passed_intersection:
                self.log_content.append(f"{self.time} car {car_name} has passed intersection. ignoring for collision prediction")
                continue
            intersection_block_time = self.intersection_block_times[car_name]
            if intersection_block_time[0] <= time_to_intersection <= intersection_block_time[1]:
                self.log_content.append(f"{self.time} Car {car.name} traveling at speed {car_edge.x_speed + car_edge.y_speed} is blocked by car {car_name} from time {intersection_block_time[0]} to {intersection_block_time[1]}")
                return True
        if not self.intersection_block_times.get(car.name):
            self.intersection_block_times[car.name] = [time_to_intersection, time_to_intersection + 5]
            # print(f"intersection reserved for {car.name} from {time_to_intersection} to {time_to_intersection+2}")
        return False

    def move_car_and_check_collision(self, car):
        car_edge = self.get_edge(car)
        car.move(car_edge.x_speed, car_edge.y_speed)
        if self.predict_collision_at_intersection(car):
            self.log_content.append(f"{self.time} Possible collision at intersection for car {car.name}, reducing speed")
            if car_edge in self.edges_with_reduced_speed:
              self.log_content.append(f"{self.time} Car {car.name} already has reduced speed, checking for collision")
              if self.predict_collision_at_intersection(car):
                  self.log_content.append(f"{self.time} Collision possible for car {car.name} after speed reduction")
                  return True
            self.edges_with_reduced_speed.append(car_edge)
            car_edge.x_speed = car_edge.x_speed - self.speed_reduction_factor if car_edge.x_speed > 0 else car_edge.x_speed
            car_edge.y_speed = car_edge.y_speed - self.speed_reduction_factor if car_edge.y_speed > 0 else car_edge.y_speed
            self.log_content.append(f"{self.time} Speed reduced for car {car.name} to {car_edge.x_speed}, {car_edge.y_speed}")
            if self.predict_collision_at_intersection(car):
              self.log_content.append(f"{self.time} Collision detected for car {car.name} after reducing speed")
              return True
        return self.check_for_collision()


    def get_edge(self, car):
        for edge in self.edges:
            if edge.is_point_on_edge(car.position):
                return edge
        raise Exception(f"Car {car.name} is not on any edge while at ({car.position.x}, {car.position.y})")

    def print_status(self, car):
      car_edge = self.get_edge(car)
      self.log_content.append(f"{self.time} car {car.name} is at {car.position.x}, {car.position.y} with speed {car_edge.x_speed}, {car_edge.y_speed}")
      if not self.car_positions.get(car.name):
          self.car_positions[car.name] = []
      self.car_positions[car.name].append(f'({car.position.x}, {car.position.y})')

    def pad_position_arrays(self):
        largest_array_size = 0
        for car in self.car_positions:
            largest_array_size = max(largest_array_size, len(self.car_positions[car]))
        for car in self.car_positions:
            if len(self.car_positions[car]) < largest_array_size:
                self.car_positions[car].extend([self.car_positions[car][-1]] * (largest_array_size - len(self.car_positions[car])))
                
    def write_log(self, filename):
        self.log_content.append(f"{self.time} Simulation ended with srf {self.speed_reduction_factor}")
        with open(filename, "w") as f:
            for line in self.log_content:
                f.write(line + "\n")
        print("Log written to", filename)
    
    def write_positions_to_excel(self):
        self.pad_position_arrays()
        df = pd.DataFrame(self.car_positions)
        columns = []
        for i in range(self.no_of_cars):
            columns.append(f"{'SE_' if not i % 2 else 'WS_'}CAR_{i}")
        df.columns = columns
        file_name = "simulation/data/car_positions.xlsx"
        df.to_excel(file_name, index=False)
        print("Positions written to file", file_name)


    def simulate(self):
        while True:
            self.active_cars = [car for car in self.cars if car.spawn_time <= self.time]
            self.log_content.append(f"{self.time} Active cars: {[car.name for car in self.active_cars]}")
            if len(self.active_cars) == 0:
                self.log_content.append(f"{self.time} All cars have left the intersection")
                break
            for car in self.active_cars:
                self.print_status(car)
                if self.move_car_and_check_collision(car):
                    return -1
                car.is_at_intersection()
                if car.visited_intersection:
                    self.cars_passed_intersection.append(car.name)
                if car.visited_intersection and car.distance_to_intersection() > 10:
                    self.cars.remove(car)
                    self.active_cars.remove(car)
                    self.log_content.append(f"{self.time} {car.name} has left the simulation")
                    if len(self.active_cars) == 0:
                        self.log_content.append(f"{self.time} All cars have left the simulation")
                        return self.time
            self.time += 1

In [445]:
# edges = [
#     # south to north road with two edges
#     Edge(Point(0, -50), Point(0, -40), 0, 1),
#     Edge(Point(0, -40), Point(0, 0), 0, 1),
#     Edge(Point(0, 0), Point(0, 40), 0, 1),

#     # west to east road with two edges
#     Edge(Point(-50, 0), Point(-40, 0), 1, 0),
#     Edge(Point(-40, 0), Point(0, 0), 1, 0),
#     Edge(Point(0, 0), Point(40, 0), 1, 0),
# ]

# edges = [
#     # south to north road with two edges
#     Edge(Point(0, -50), Point(0, 40), 0, 1),

#     # west to east road with two edges
#     Edge(Point(-50, 0), Point(40, 0), 1, 0),
# ]

# cars = [
#     Car("A", Point(0, -50)),
#     Car("B", Point(-50, 0)),
# ]

In [446]:
import numpy as np

class QLearningController(Controller):
    def __init__(self, cars, edges, alpha=0.1, gamma=0.6, epsilon=0.1):
        super().__init__(cars, edges)
        self.q_table = np.zeros((101, 101))  # Q-table for speed_reduction_factor values from 0 to 1
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate

    def update_q_table(self):
      speed_reduction_factors = np.arange(0, 1.01, 0.01)
      no_of_srfs = len(speed_reduction_factors)
      self.q_table = np.zeros((no_of_srfs, no_of_srfs))
      for i in range(no_of_srfs):
        reward = self.simulate_with_speed_reduction(speed_reduction_factors[i])
        for j in range(no_of_srfs):
            # print("reward", reward, "for", i/100)
            next_max = np.max(self.q_table[i])
            self.q_table[i, j] = (1 - self.alpha) * self.q_table[i, j] + self.alpha * (reward + self.gamma * next_max)


    def simulate_with_speed_reduction(self, speed_reduction_factor):
      if speed_reduction_factor == 1:
        return -1000
      controller_copy = self.copy_controller(speed_reduction_factor)
      controller_copy.speed_reduction_factor = speed_reduction_factor
      total_time_taken = controller_copy.simulate()
      controller_copy.write_log("logs/training_logs/srf_" + str(speed_reduction_factor) + ".log")
      if total_time_taken == -1:
          return -1000  # Penalize collision heavily
      return -total_time_taken  # Negative total time taken to minimize delay

    def copy_controller(self, speed_reduction_factor):
        cars_copy = [Car(car.name, Point(car.position.x, car.position.y), car.source, car.destination, car.spawn_time) for car in self.cars]
        edges_copy = [Edge(edge.p1, edge.p2, edge.x_speed, edge.y_speed) for edge in self.edges]
        return Controller(cars_copy, edges_copy, speed_reduction_factor)

# Example usage
edges = [
    Edge(Point(0, -50), Point(0, 40), 0, 1),  # South to north road
    Edge(Point(-50, 0), Point(40, 0), 1, 0),  # West to east road
    Edge(Point(1, -50), Point(1, 40), 0, -1),  # North to south road
    Edge(Point(-50, 1), Point(40, 1), 1, 0),  # East to west road
]
cars = [
    Car("A", Point(0, -50), 'S', 'E', 0),
    Car("B", Point(-50, 0), 'E', 'S', 0),
    Car("C", Point(0, -50), 'S', 'E', 25),
    Car("D", Point(-50, 0), 'E', 'S', 28),
    Car("E", Point(0, -50), 'S', 'E', 32),
    Car("F", Point(-50, 0), 'E', 'S', 35),
]

controller = QLearningController(cars, edges)
total_time_taken = controller.update_q_table()
q_table = controller.q_table

Log written to logs/training_logs/srf_0.0.log
Log written to logs/training_logs/srf_0.01.log
Log written to logs/training_logs/srf_0.02.log
Log written to logs/training_logs/srf_0.03.log
Log written to logs/training_logs/srf_0.04.log
Log written to logs/training_logs/srf_0.05.log
Log written to logs/training_logs/srf_0.06.log
Log written to logs/training_logs/srf_0.07.log
Log written to logs/training_logs/srf_0.08.log
Log written to logs/training_logs/srf_0.09.log
Log written to logs/training_logs/srf_0.1.log
Log written to logs/training_logs/srf_0.11.log
Log written to logs/training_logs/srf_0.12.log
Log written to logs/training_logs/srf_0.13.log
Log written to logs/training_logs/srf_0.14.log
Log written to logs/training_logs/srf_0.15.log
Log written to logs/training_logs/srf_0.16.log
Log written to logs/training_logs/srf_0.17.log
Log written to logs/training_logs/srf_0.18.log
Log written to logs/training_logs/srf_0.19.log
Log written to logs/training_logs/srf_0.2.log
Log written to l

In [447]:
max_q_values = []
for i in range(len(q_table)):
  max_q_values.append(max(q_table[i]))
most_optimal_srf = np.argmax(max_q_values)/100
print("most optimal edge weight", 1-most_optimal_srf, "srf", most_optimal_srf)

most optimal edge weight 0.84 srf 0.16


In [448]:
optimal_controller = Controller(cars, edges, most_optimal_srf)
delay = optimal_controller.simulate()
optimal_controller.write_log("logs/positions.log")
optimal_controller.write_positions_to_excel()
delay

Log written to logs/positions.log
Positions written to file simulation/data/car_positions.xlsx


104